In [1]:
import json
import requests
from src import utils
import time
import os

In [2]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api.json'

In [3]:
api = utils.read_json_file(api_path)

In [4]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
# in_path = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-train.jsonl'

In [5]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [6]:
os.makedirs(output_path, exist_ok=True)
output_file_name = 'result-' + input_file.split('/')[-1]
output_file = output_path + '/' + output_file_name
print(f'output_file : {output_file}')

output_file : /userhomes/philhoon/kt-ai-challenge/result/result-openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl


In [7]:
jsonl_writer = utils.JSONLWriter(output_file)

In [8]:
data = utils.read_jsonlines(input_file)

In [9]:
# file_name

In [10]:
len(data)

1319

In [11]:
# api

In [12]:
# url = api['url']

In [13]:
# headers = api['headers']

In [14]:
# question = "질문 : 파운드 스털링은 가치가 있나요? 용어설명 :  사실목록 : ['1파운드는 법정 화폐입니다.', '법정 화폐는 정부 법령에 의해 뒷받침되며 내재적 가치가 없습니다', '2020년 5월 기준 1파운드 스털링은 약 1.24달러의 가치가 있습니다.']. 사실목록을 바탕으로 질문에 대한 답변을 사실 또는 거짓으로 대답하세요"/

In [15]:
# headers

In [16]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [17]:
model = model_api(api['url'], api['headers'])

In [18]:
BODY_TEMP = {
    "serviceInstanceId" : "ddm3kz9g", 
    "nluType" : "009",
    "apiType" : "002", 
    "utterance" : None
}

In [19]:
# body = json.dumps(body_temp)

In [20]:
# print(body)

In [21]:
# response = api.generate(body)

In [22]:
# context = response.json()['data'][0]['result'][0]

In [23]:
# print(context)

In [24]:
def get_gsm8k_zs(que):
    prompt = f'질문 : {que} \n정답 : '
    return prompt

In [25]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [26]:
# from pprint import pprint

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [ ]:
cnt = 0
split_cnt = 0
for ins in data:
    kor_que = ins['kor_question']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue

    kor_ans = ins['kor_answer']
    prompt = get_gsm8k_zs(kor_que)
    
    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print('----')
    
    ins['kor_infer'] = res_context
    jsonl_writer.write_json_line(ins)
    
#     print(prompt)
    if cnt == 5:
        break
    else:
        cnt += 1